In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1005. Miscellaneous Analyses/P1005.7. EV Proteomics/P1005.7.3. Analyses/P1005.7.3.2. Group Distribution Analysis/Output')
cur_date = "013124"

library(readxl)
library(tidyverse)
library(reshape2)
library(openxlsx)

# reading in file
proteomics_df = data.frame(read_excel("Input/Imputed_Proteomics_Data_013024.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(proteomics_df)

,ID,Treatment,Protein,Value
,<chr>,<chr>,<chr>,<dbl>
1,Control_R1,Control,P00761,48817715
2,Control_R1,Control,P60709,15596874
3,Control_R1,Control,P0C0S8.Q96KK5.Q99878.Q9BTM1,2070354
4,Control_R1,Control,P23527,1429094
5,Control_R1,Control,P68104,13350056
6,Control_R1,Control,P06733,9269053


In [3]:
# splitting dfs for analyses and adding a log2 value
proteomics_df$log2Value = log2(proteomics_df$Value)

cev_df = proteomics_df %>%
    filter(Treatment == 'Control' | Treatment == 'CEV')
pev_df = proteomics_df %>%
    filter(Treatment == 'Control' | Treatment == 'PEV')
pfas_df = proteomics_df %>%
    filter(Treatment == 'Control' | Treatment == 'PFAS')
pfas_cev_df = proteomics_df %>%
    filter(Treatment == 'Control' | Treatment == 'PFAS.CEV')
pfas_pev_df = proteomics_df %>%
    filter(Treatment == 'Control' | Treatment == 'PFAS.PEV')

head(cev_df)

,ID,Treatment,Protein,Value,log2Value
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Control_R1,Control,P00761,48817715,25.54090
2,Control_R1,Control,P60709,15596874,23.89475
3,Control_R1,Control,P0C0S8.Q96KK5.Q99878.Q9BTM1,2070354,20.98145
4,Control_R1,Control,P23527,1429094,20.44667
5,Control_R1,Control,P68104,13350056,23.67034
6,Control_R1,Control,P06733,9269053,23.14399


In [4]:
unique(proteomics_df$Treatment)

[1] "Control"  "CEV"      "PEV"      "PFAS"     "PFAS.CEV" "PFAS.PEV"

Typically, normality and homogeneity of variances would be tested for first, however a non-parametric test (Wilcoxon rank sum) will be used given the small sample size (n = 6) to determine if there are statistically significant distribution differences in control and treatment groups.

In [5]:
wilcox_test_values = function(df){
    # """
    # Running wilcoxon rank sum tests after filtering for set, treatment, exposure, and protein using a loop. 
    # Ultimately using this test to compare proteins (control vs. treatment).

    # :param: subsetted dataframe, empty dataframe
    # :output: a dataframe containing the treatment, protein, u stat, p value, p adj

    # """
    proteins = unique(df$Protein)
    treatments = unique(df$Treatment)[2:6]
    
    values_df = data.frame()
    
    # iterating through each tx and protein
    for (i in 1:length(treatments)){
        for(j in 1:length(proteins)){
                # control df
                control_df = df %>%
                    filter(Treatment == 'Control', Protein == proteins[j])
    
                # treatment df
                treatment_df = df %>%
                    filter(Treatment == treatments[i], Protein == proteins[j])
    
                # wilcox test
                wilcox_test = wilcox.test(control_df$log2Value, treatment_df$log2Value)
                
                # calculating FC to get directionality
                FC = log2(mean(treatment_df$Value)/mean(control_df$Value))
    
                # contains tx, protein, u stat, and p value
                values_vector = cbind(treatments[i], proteins[j], FC, wilcox_test$statistic, wilcox_test$p.value)
                values_df = rbind(values_df, values_vector)
         }
    }
  
    # adding col names
    colnames(values_df) = c("Treatment", "Protein", "log2FC", "Statistic", "P Value")
    
   # calculating padj values
    values_df = values_df %>%
        group_by(Treatment) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    values_df$log2FC = as.numeric(values_df$log2FC) 
    values_df$Statistic = as.numeric(values_df$Statistic)
    values_df$`P Value` = as.numeric(values_df$`P Value`) 
    
    return(values_df)
}

# calling fn
wilcox_df = wilcox_test_values(proteomics_df)
head(wilcox_df)

Treatment,Protein,log2FC,Statistic,P Value,P Adj
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
CEV,P00761,-0.22382871,22,0.5887446,1
CEV,P60709,0.03730957,15,0.6991342,1
CEV,P0C0S8.Q96KK5.Q99878.Q9BTM1,-0.05177057,20,0.8181818,1
CEV,P23527,-0.04699730,17,0.9372294,1
CEV,P68104,0.04958492,11,0.3095238,1
CEV,P06733,-0.01960391,25,0.3095238,1


In [6]:
# determining number of sig genes based on p adj < 0.05
wilcox_df %>%
    group_by(Treatment) %>%
    filter(`P Adj` < 0.05) %>%
    count()

Treatment,n
<chr>,<int>
PFAS,4253
PFAS.CEV,4858
PFAS.PEV,4737


In [7]:
# exporting
write.xlsx(wilcox_df, paste0(Output,"/", "Proteomics_Statistical_Results", cur_date, ".xlsx"), 
           rowNames = FALSE)